# Step 3: Multi-Method Training
## Fixing the Two Diagnosed Weaknesses from Step 2

**Step 2 result:** EfficientNet-B0 → FF++ Val AUC=0.7241, Celeb-DF AUC=0.6659

**Two diagnosed problems:**
1. Trained on Deepfakes only → learned one method's fingerprint, not general semantics
2. Only 300 videos → overfitting (train loss 0.25, val loss 0.85 by epoch 20)

**This notebook fixes both:**
- All 4 FF++ methods: Deepfakes + Face2Face + FaceSwap + NeuralTextures
- 600 real + 600 fake videos (150 per method)
- Fixed double forward-pass bug from Step 2 train loop
- Everything else identical to Step 2

**Expected improvement:** +8-13% Celeb-DF AUC (0.6659 → 0.75-0.80)

## Section 1 — Setup

In [ ]:
import os, json, random, time, warnings, sys
from pathlib import Path
from typing import Optional
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
warnings.filterwarnings('ignore')

SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device : {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU    : {torch.cuda.get_device_name(0)}")
    print(f"VRAM   : {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")

OUTPUT_DIR = Path('/kaggle/working/step3')
CKPT_DIR   = OUTPUT_DIR / 'checkpoints'
PLOTS_DIR  = OUTPUT_DIR / 'plots'
for d in [OUTPUT_DIR, CKPT_DIR, PLOTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)
print(f"Outputs → {OUTPUT_DIR}")


In [ ]:
CFG = {
    'img_size':        224,
    'n_frames':        4,       # frames per video
    'n_train_real':    600,     # ↑ was 300
    'n_train_fake':    600,     # ↑ was 300 (150 per method × 4 methods)
    'n_val_each':      50,      # per class for validation
    'epochs':          20,
    'batch_size':      32,
    'lr':              1e-4,
    'weight_decay':    1e-4,
    'warmup_epochs':   3,
    'dropout':         0.3,
    'label_smoothing': 0.0,     # CONFIRMED: must be 0.0
}

TRAIN_METHODS = ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']

print("Config:")
for k, v in CFG.items():
    print(f"  {k:22s}: {v}")
print(f"Train methods: {TRAIN_METHODS}")


## Section 2 — Dataset Paths & Splits

In [ ]:
KAGGLE_INPUT = Path('/kaggle/input')

def locate_ff_root(base):
    known = base / 'datasets' / 'xdxd003' / 'ff-c23' / 'FaceForensics++_C23'
    if known.exists(): return known
    for d in sorted(base.rglob('*')):
        if d.is_dir():
            if sum(1 for m in ['Deepfakes','Face2Face','FaceSwap'] if (d/m).exists()) >= 2:
                return d
    return None

def locate_celeb_root(base):
    known = base / 'datasets' / 'reubensuju' / 'celeb-df-v2'
    if known.exists(): return known
    for d in sorted(base.rglob('*')):
        if d.is_dir() and (d/'Celeb-real').exists(): return d
    return None

FF_ROOT    = locate_ff_root(KAGGLE_INPUT)
CELEB_ROOT = locate_celeb_root(KAGGLE_INPUT)
print(f"FF++    : {FF_ROOT}")
print(f"Celeb-DF: {CELEB_ROOT}")

FF_REAL = sorted(FF_ROOT.rglob('original*/*.mp4')) if FF_ROOT else []
if not FF_REAL and FF_ROOT:
    FF_REAL = sorted(p for p in FF_ROOT.rglob('*.mp4') if 'original' in str(p).lower())

FF_FAKE_BY_METHOD = {}
for method in TRAIN_METHODS:
    paths = sorted((FF_ROOT/method).glob('*.mp4')) if FF_ROOT and (FF_ROOT/method).exists() else []
    FF_FAKE_BY_METHOD[method] = paths
    print(f"  FF++/{method:20s}: {len(paths)} videos")
print(f"  FF++/{'real':20s}: {len(FF_REAL)} videos")

CDF_REAL, CDF_FAKE = [], []
if CELEB_ROOT:
    CDF_REAL = (sorted((CELEB_ROOT/'Celeb-real').glob('*.mp4')) +
                sorted((CELEB_ROOT/'YouTube-real').glob('*.mp4')))
    CDF_FAKE = sorted((CELEB_ROOT/'Celeb-synthesis').glob('*.mp4'))
    print(f"  Celeb-DF real: {len(CDF_REAL)} | fake: {len(CDF_FAKE)}")


In [ ]:
rng = random.Random(SEED)

# ── Extract video ID from filename ────────────────────────────────────────────
# FF++ filenames: "000_003.mp4" → ID is "000" (source video)
def get_video_id(path):
    return Path(path).stem.split('_')[0]

# ── Build ID-level splits FIRST ───────────────────────────────────────────────
all_ids = sorted(set(get_video_id(p) for p in FF_REAL))
rng.shuffle(all_ids)

n_train_ids = int(len(all_ids) * 0.75)   # 75% train, 25% val
train_ids   = set(all_ids[:n_train_ids])
val_ids     = set(all_ids[n_train_ids:])

print(f"Total video IDs: {len(all_ids)}")
print(f"Train IDs: {len(train_ids)} | Val IDs: {len(val_ids)}")
print(f"(No ID appears in both — guaranteed no content leakage)")

# ── Training set ──────────────────────────────────────────────────────────────
train_real = [p for p in FF_REAL if get_video_id(p) in train_ids]
train_real = rng.sample(train_real, min(CFG['n_train_real'], len(train_real)))
TRAIN_DATA = [(p, 0) for p in train_real]

n_per_method = CFG['n_train_fake'] // len(TRAIN_METHODS)
for method in TRAIN_METHODS:
    pool   = [p for p in FF_FAKE_BY_METHOD[method] if get_video_id(p) in train_ids]
    picked = rng.sample(pool, min(n_per_method, len(pool)))
    TRAIN_DATA += [(p, 1) for p in picked]
rng.shuffle(TRAIN_DATA)

# ── Validation set ────────────────────────────────────────────────────────────
val_real = [p for p in FF_REAL if get_video_id(p) in val_ids]
val_real = rng.sample(val_real, min(CFG['n_val_each'], len(val_real)))
VAL_DATA = [(p, 0) for p in val_real]

for method in TRAIN_METHODS:
    pool   = [p for p in FF_FAKE_BY_METHOD[method] if get_video_id(p) in val_ids]
    picked = rng.sample(pool, min(CFG['n_val_each'] // len(TRAIN_METHODS), len(pool)))
    VAL_DATA += [(p, 1) for p in picked]
rng.shuffle(VAL_DATA)

# ── Celeb-DF ──────────────────────────────────────────────────────────────────
n_cdf    = min(200, len(CDF_REAL), len(CDF_FAKE))
CDF_TEST = ([(p, 0) for p in rng.sample(CDF_REAL, n_cdf)] +
            [(p, 1) for p in rng.sample(CDF_FAKE, n_cdf)])

print(f"\nTrain: {sum(1 for _,l in TRAIN_DATA if l==0)} real + "
      f"{sum(1 for _,l in TRAIN_DATA if l==1)} fake = {len(TRAIN_DATA)}")
print(f"Val  : {sum(1 for _,l in VAL_DATA if l==0)} real + "
      f"{sum(1 for _,l in VAL_DATA if l==1)} fake = {len(VAL_DATA)}")
print(f"CDF  : {n_cdf} real + {n_cdf} fake = {len(CDF_TEST)}")

## Section 3 — Dataset (Frame Pre-Extraction)

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

train_tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.05),
    transforms.RandomGrayscale(p=0.05),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])
val_tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])


def load_frames(video_path, n_frames, img_size):
    """Extract n evenly-spaced frames from video. Returns list of uint8 arrays or None."""
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened(): return None
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total < 1:
        cap.release(); return None
    positions = np.linspace(0, total - 1, n_frames, dtype=int)
    frames = []
    for pos in positions:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(pos))
        ret, frame = cap.read()
        if not ret: continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w = frame.shape[:2]
        frame = frame[int(h*0.05):int(h*0.95), int(w*0.10):int(w*0.90)]
        frame = cv2.resize(frame, (img_size, img_size))
        frames.append(frame)
    cap.release()
    if not frames: return None
    while len(frames) < n_frames: frames.append(frames[-1])
    return frames[:n_frames]


class DeepfakeDataset(Dataset):
    """Pre-extracts all frames at construction. DataLoader does only transforms."""
    def __init__(self, video_label_pairs, transform, n_frames, img_size):
        self.transform = transform
        self.items = []
        failed = 0
        for path, label in tqdm(video_label_pairs, ncols=80, desc='Loading'):
            frames = load_frames(str(path), n_frames, img_size)
            if frames is None:
                failed += 1
                continue
            for f in frames:
                self.items.append((f, label))
        print(f"  {len(self.items)} frames ready, {failed} videos failed")

    def __len__(self): return len(self.items)

    def __getitem__(self, idx):
        frame, label = self.items[idx]
        return self.transform(frame), torch.tensor(label, dtype=torch.long)


print("Pre-extracting frames (this takes ~5 min)...")
t0 = time.time()
train_ds = DeepfakeDataset(TRAIN_DATA, train_tf, CFG['n_frames'], CFG['img_size'])
val_ds   = DeepfakeDataset(VAL_DATA,   val_tf,   CFG['n_frames'], CFG['img_size'])
cdf_ds   = DeepfakeDataset(CDF_TEST,   val_tf,   CFG['n_frames'], CFG['img_size'])
print(f"Done in {time.time()-t0:.1f}s")

train_loader = DataLoader(train_ds, batch_size=CFG['batch_size'],
                          shuffle=True,  num_workers=0, pin_memory=False)
val_loader   = DataLoader(val_ds,   batch_size=CFG['batch_size'],
                          shuffle=False, num_workers=0, pin_memory=False)
cdf_loader   = DataLoader(cdf_ds,   batch_size=CFG['batch_size'],
                          shuffle=False, num_workers=0, pin_memory=False)

print(f"Train frames: {len(train_ds)} | Val: {len(val_ds)} | CDF: {len(cdf_ds)}")
x, y = next(iter(train_loader))
print(f"Batch: x={x.shape}, labels={y.unique().tolist()}")


## Section 4 — Model

In [ ]:
class DeepfakeDetector(nn.Module):
    def __init__(self, dropout=CFG['dropout']):
        super().__init__()
        self.backbone = models.efficientnet_b0(
            weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        in_feat = self.backbone.classifier[1].in_features  # 1280
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(in_feat, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout * 0.5),
            nn.Linear(256, 2),
        )
        self.backbone_params = list(self.backbone.features.parameters())
        self.head_params     = list(self.backbone.classifier.parameters())

    def forward(self, x):
        return self.backbone(x)

    def get_param_groups(self, base_lr):
        return [
            {'params': self.backbone_params, 'lr': base_lr / 10},  # 1e-5
            {'params': self.head_params,     'lr': base_lr},        # 1e-4
        ]

model = DeepfakeDetector().to(DEVICE)
print(f"EfficientNet-B0: {sum(p.numel() for p in model.parameters())/1e6:.2f}M params")

with torch.no_grad():
    out = model(torch.randn(2, 3, 224, 224).to(DEVICE))
    print(f"Forward pass: (2,3,224,224) → {out.shape} ✓")


## Section 5 — Training

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=CFG['label_smoothing'])
optimizer = torch.optim.AdamW(model.get_param_groups(CFG['lr']),
                               weight_decay=CFG['weight_decay'])

def lr_lambda(epoch):
    if epoch < CFG['warmup_epochs']:
        return (epoch + 1) / CFG['warmup_epochs']
    progress = (epoch - CFG['warmup_epochs']) / max(1, CFG['epochs'] - CFG['warmup_epochs'])
    return 0.5 * (1 + np.cos(np.pi * progress))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)


def train_epoch(model, loader):
    model.train()
    total_loss, correct, total = 0.0, 0, 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(x)               # ONE forward pass — fixed double-call bug
        loss   = criterion(logits, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()
        correct    += (logits.detach().argmax(1) == y).sum().item()
        total      += y.size(0)
    return total_loss / len(loader), correct / total


def evaluate(model, loader):
    model.eval()
    all_labels, all_probs = [], []
    total_loss, n_batches = 0.0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y    = x.to(DEVICE), y.to(DEVICE)
            logits  = model(x)
            total_loss += criterion(logits, y).item()
            probs   = F.softmax(logits, dim=1)[:, 1]
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
            n_batches += 1
    labels = np.array(all_labels)
    probs  = np.array(all_probs)
    auc    = roc_auc_score(labels, probs) if len(np.unique(labels)) > 1 else 0.5
    acc    = ((probs > 0.5).astype(int) == labels).mean()
    return {'auc': auc, 'acc': acc, 'loss': total_loss / max(n_batches, 1),
            'labels': labels, 'probs': probs}

print("✅ train_epoch and evaluate ready")
print(f"   Steps per epoch: {len(train_loader)}")


In [ ]:
history = {'train_loss':[], 'train_acc':[], 'val_loss':[], 'val_auc':[], 'lr':[]}
best_val_auc, best_epoch = 0.0, 0
start_time = time.time()

print("=" * 68)
print(f"{'Ep':>3} {'TrLoss':>8} {'TrAcc':>7} {'VaLoss':>8} "
      f"{'VaAUC':>7} {'VaAcc':>7} {'LR':>9} {'t':>5}")
print("=" * 68)

for epoch in range(CFG['epochs']):
    t0 = time.time()

    tr_loss, tr_acc = train_epoch(model, train_loader)
    val_m           = evaluate(model, val_loader)
    scheduler.step()
    lr = optimizer.param_groups[1]['lr']   # head LR

    history['train_loss'].append(tr_loss)
    history['train_acc'].append(tr_acc)
    history['val_loss'].append(val_m['loss'])
    history['val_auc'].append(val_m['auc'])
    history['lr'].append(lr)

    flag = ' ✓' if val_m['auc'] > best_val_auc else ''
    print(f"{epoch+1:>3} {tr_loss:>8.4f} {tr_acc:>7.3f} {val_m['loss']:>8.4f} "
          f"{val_m['auc']:>7.4f} {val_m['acc']:>7.3f} {lr:>9.2e} "
          f"{time.time()-t0:>4.0f}s{flag}")
    sys.stdout.flush()

    if val_m['auc'] > best_val_auc:
        best_val_auc = val_m['auc']
        best_epoch   = epoch + 1
        torch.save({'epoch': epoch, 'model_state': model.state_dict(),
                    'val_auc': best_val_auc, 'cfg': CFG},
                   CKPT_DIR / 'best.pth')

total_time = time.time() - start_time
print("=" * 68)
print(f"Best val AUC : {best_val_auc:.4f} at epoch {best_epoch}")
print(f"Total time   : {total_time/60:.1f} min")


## Section 6 — Evaluation & Comparison

In [ ]:
ckpt = torch.load(CKPT_DIR / 'best.pth', map_location=DEVICE, weights_only=False)
model.load_state_dict(ckpt['model_state'])
print(f"Loaded best model — epoch {ckpt['epoch']+1}, val AUC={ckpt['val_auc']:.4f}")

ff_m  = evaluate(model, val_loader)
cdf_m = evaluate(model, cdf_loader)

STEP2 = {'ff_auc': 0.7241, 'cdf_auc': 0.6659}

print("\n" + "=" * 58)
print("STEP 2 vs STEP 3 COMPARISON")
print("=" * 58)
print(f"{'Metric':<32} {'Step 2':>9} {'Step 3':>9} {'Delta':>7}")
print("-" * 58)
print(f"{'FF++ Val AUC':<32} {STEP2['ff_auc']:>9.4f} {ff_m['auc']:>9.4f} "
      f"{ff_m['auc']-STEP2['ff_auc']:>+7.4f}")
print(f"{'Celeb-DF AUC (cross-dataset)':<32} {STEP2['cdf_auc']:>9.4f} {cdf_m['auc']:>9.4f} "
      f"{cdf_m['auc']-STEP2['cdf_auc']:>+7.4f}")
gap2 = STEP2['ff_auc'] - STEP2['cdf_auc']
gap3 = ff_m['auc'] - cdf_m['auc']
print(f"{'Generalization gap':<32} {gap2:>9.4f} {gap3:>9.4f} {gap3-gap2:>+7.4f}")
print("=" * 58)

if cdf_m['auc'] >= 0.75:
    verdict = "🟢 STRONG — Ready to build V8.0 temporal module on RunPod"
elif cdf_m['auc'] >= 0.68:
    verdict = "🟡 GOOD — Clear improvement. B4 backbone will push further"
else:
    verdict = "🟡 MODERATE — Improvement but weaker than expected"
print(f"\n{verdict}")


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Step 3: Multi-Method Training — Results', fontsize=14, fontweight='bold')

x = range(1, len(history['train_loss']) + 1)

# Loss
axes[0].plot(x, history['train_loss'], label='Train', color='#3498db', linewidth=2)
axes[0].plot(x, history['val_loss'],   label='Val',   color='#e74c3c', linewidth=2)
axes[0].axhline(0.693, color='gray', linestyle=':', alpha=0.6, label='Random (0.693)')
axes[0].set_title('Loss'); axes[0].set_xlabel('Epoch')
axes[0].legend(); axes[0].grid(True, alpha=0.3)

# AUC with Step 2 reference lines
axes[1].plot(x, history['val_auc'], color='#2ecc71', linewidth=2.5, label='Step 3 Val AUC')
axes[1].axhline(best_val_auc,     color='#2ecc71', linestyle='--', alpha=0.6,
                label=f'Step3 best={best_val_auc:.4f}')
axes[1].axhline(STEP2['ff_auc'], color='gray',    linestyle='--', alpha=0.5,
                label=f'Step2 val={STEP2["ff_auc"]:.4f}')
axes[1].axhline(cdf_m['auc'],   color='#e74c3c', linestyle='--', alpha=0.8,
                label=f'CDF AUC={cdf_m["auc"]:.4f}')
axes[1].axhline(STEP2['cdf_auc'], color='#e74c3c', linestyle=':', alpha=0.5,
                label=f'Step2 CDF={STEP2["cdf_auc"]:.4f}')
axes[1].set_title('Val AUC'); axes[1].set_xlabel('Epoch')
axes[1].set_ylim(0.40, 1.0); axes[1].legend(fontsize=8); axes[1].grid(True, alpha=0.3)

# ROC curves
for color, m, label in [
    ('#3498db', ff_m,  f"FF++ Val  (AUC={ff_m['auc']:.4f})"),
    ('#e74c3c', cdf_m, f"Celeb-DF  (AUC={cdf_m['auc']:.4f})"),
]:
    fpr, tpr, _ = roc_curve(m['labels'], m['probs'])
    axes[2].plot(fpr, tpr, color=color, linewidth=2, label=label)
axes[2].plot([0,1],[0,1],'k--', alpha=0.4, label='Random')
axes[2].set_title('ROC Curves'); axes[2].set_xlabel('FPR'); axes[2].set_ylabel('TPR')
axes[2].legend(fontsize=9); axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'step3_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ step3_results.png")


## Section 7 — Save Results

In [ ]:
results = {
    'model':               'EfficientNet-B0 ImageNet pretrained',
    'train_methods':       TRAIN_METHODS,
    'n_train_videos':      len(TRAIN_DATA),
    'best_epoch':          best_epoch,
    'ff_val':              {'auc': round(ff_m['auc'],  4), 'acc': round(ff_m['acc'],  4)},
    'celeb_df':            {'auc': round(cdf_m['auc'], 4), 'acc': round(cdf_m['acc'], 4)},
    'gap':                 round(ff_m['auc'] - cdf_m['auc'], 4),
    'step2_cdf_auc':       STEP2['cdf_auc'],
    'improvement':         round(cdf_m['auc'] - STEP2['cdf_auc'], 4),
    'training_minutes':    round(total_time / 60, 1),
}

with open(OUTPUT_DIR / 'step3_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("=" * 58)
print("STEP 3 COMPLETE — V8.0 ROADMAP")
print("=" * 58)
print(f"  Step 2 (1 method, 300 vids) : CDF AUC = {STEP2['cdf_auc']:.4f}")
print(f"  Step 3 (4 methods, 600 vids): CDF AUC = {cdf_m['auc']:.4f}")
print(f"  Improvement                 : {results['improvement']:+.4f}")
print()
print("Remaining improvements for V8.0 on RunPod A100:")
print("  [ ] EfficientNet-B4 backbone   (+3-5% expected)")
print("  [ ] Full FF++ training set     (+2-4% expected)")
print("  [ ] Temporal Mamba module      (+5-10% expected, main contribution)")
print(f"\n  Current floor : {cdf_m['auc']:.4f}")
print(f"  V8.0 target   : 0.90+")
print(f"  SOTA          : 0.9629 (WMamba)")
print(f"\n✅ Results → {OUTPUT_DIR / 'step3_results.json'}")
